In [8]:
from math import ceil

import pandas as pd

from ddeserts import read_cvap_csv, moe_of_sum, moe_of_ratio

In [2]:
# read in list of charter cities
charter_cities_path = 'data/cacities/charter-cities.txt'

with open(charter_cities_path) as f:
    charter_cities = { line.strip() for line in f }

In [3]:
# load the data
places_path = 'data/census/CVAP_2015-2019_ACS_csv_files/Place.csv'

pre_filter = lambda line: 'California' in line and 'Total' in line
filter = lambda row: row['state'] == 'California' and row['geotype'] != 'cdp'

ca_city_rows = list(read_cvap_csv(places_path, pre_filter=pre_filter, filter=filter))


df = pd.DataFrame.from_records(ca_city_rows, index=['table', 'line'])
df['tot_moe'] = df['tot_moe'].astype('int')
df['adu_moe'] = df['adu_moe'].astype('int')
df['cvap_moe'] = df['cvap_moe'].astype('int')
df['cit_moe'] = df['cit_moe'].astype('int')


In [15]:
# add extra columns
df['has_charter'] = df['name'].isin(charter_cities)

df['dvap_est'] = df['adu_est'] - df['cvap_est']
df['dvap_moe'] = df.apply(lambda r: ceil(moe_of_sum(r['adu_moe'], r['cvap_moe'])), axis=1).astype('int')
df['dvap_pct'] = df['dvap_est'] / df['adu_est']
df['dvap_pct_moe'] = df.apply(lambda r: moe_of_ratio(
    r['dvap_est'], r['dvap_moe'], r['adu_est'], r['adu_moe']), axis=1).astype('float')


In [21]:
dvap = df.loc[df['has_charter']][
    ['name', 'adu_est', 'adu_moe', 'dvap_est', 'dvap_moe', 'dvap_pct', 'dvap_pct_moe']].sort_values('dvap_pct')
dvap[20:40]

name  adu_est  adu_moe  dvap_est  dvap_moe  dvap_pct  \
table line                                                                      
Place 33060  Huntington Beach   161815      973     10710      1784  0.066187   
      36050        Marysville     9415      302       655       473  0.069570   
      39625     Redondo Beach    52620      654      3790       962  0.072026   
      39456     Rancho Mirage    16820      283      1220       484  0.072533   
      41770           Solvang     4665      160       350       291  0.075027   
      40483           Ventura    86525      731      6820      1367  0.078821   
      44370          Whittier    66350      574      5255       905  0.079201   
      34542         La Quinta    33055      614      2645       996  0.080018   
      34815           Lemoore    18985      468      1525       689  0.080327   
      26547     Big Bear Lake     4190      147       360       250  0.085919   
      36778          Monterey    24015      450      2180       808  0.090777   
      31253            Folsom    59245      611      5505      1141  0.092919   
      43707           Visalia    93190     1089      8835      1827  0.094806   
      25221           Alameda    62565      524      5990       983  0.095740   
      41003      Santa Monica    77880      719      7630      1198  0.097971   
      38234       Palm Desert    45015      544      4440      1080  0.098634   
      40951        Santa Cruz    56275      559      5615      1022  0.099778   
      38624          Petaluma    47985      507      4810       867  0.100240   
      41562       Signal Hill     9355      300       955       479  0.102084   
      28198          Cerritos    40755      445      4230       861  0.103791   

             dvap_pct_moe  
table line                 
Place 33060      0.011492  
      36050      0.054209  
      39625      0.019419  
      39456      0.030509  
      41770      0.067260  
      40483      0.016605  
      44370      0.014450  
      34542      0.032216  
      34815      0.039239  
      26547      0.064959  
      36778      0.036022  
      31253      0.020428  
      43707      0.020958  
      25221      0.016653  
      41003      0.016439  
      38234      0.025492  
      40951      0.019344  
      38624      0.019332  
      41562      0.056281  
      28198      0.022505

In [19]:
dvap = df.loc[df['has_charter']][
    ['name', 'adu_est', 'adu_moe', 'dvap_est', 'dvap_moe', 'dvap_pct', 'dvap_pct_moe']].sort_values('dvap_est')
dvap[-30:]

name  adu_est  adu_moe  dvap_est  dvap_moe  dvap_pct  \
table line                                                                    
Place 28900         Compton    68500      942     17275      1551  0.252190   
      40795       San Mateo    82505      817     17400      1420  0.210896   
      36583         Modesto   158385     1042     17825      1798  0.112542   
      37077   Mountain View    65625      660     19675      1403  0.299810   
      38442        Pasadena   115505      864     19685      1731  0.170426   
      41042      Santa Rosa   140870      836     20410      1779  0.144885   
      39742        Richmond    85930      921     20620      1923  0.239963   
      40990     Santa Maria    72750      872     26085      1627  0.358557   
      32618         Hayward   125155      887     27275      1738  0.217930   
      39105          Pomona   114115      947     27900      1724  0.244490   
      40457  San Bernardino   152070     1556     28590      2558  0.188006   
      40925     Santa Clara   102390      787     29365      1797  0.286796   
      31903        Glendale   164850      999     33510      2044  0.203276   
      39937       Riverside   248645     1714     35775      2635  0.143880   
      26040     Bakersfield   264830     1753     36715      2963  0.138636   
      42199        Stockton   223645     1052     36910      2403  0.165038   
      28432     Chula Vista   200870     1391     37005      2888  0.184224   
      40340         Salinas   107810      882     42475      1842  0.393980   
      42407       Sunnyvale   120715      822     42505      1930  0.352110   
      40301      Sacramento   385200     1454     44720      2563  0.116096   
      33346          Irvine   212955     1454     47620      3036  0.223615   
      31617          Fresno   375795     1548     52895      3111  0.140755   
      35309      Long Beach   362340     1852     54060      3234  0.149197   
      37740         Oakland   340625     1274     55755      2591  0.163684   
      25546         Anaheim   265820     1596     66135      3423  0.248796   
      40899       Santa Ana   243370     1502     83035      2932  0.341188   
      40587   San Francisco   757415        0    104965      2317  0.138583   
      40535       San Diego  1129095     2591    157595      5312  0.139576   
      40665        San Jose   797090     1566    164515      4078  0.206395   
      35400     Los Angeles  3145520     3613    728480      8499  0.231593   

             dvap_pct_moe  
table line                 
Place 28900      0.026474  
      40795      0.019492  
      36583      0.012173  
      37077      0.024642  
      38442      0.016384  
      41042      0.013569  
      39742      0.025221  
      40990      0.026985  
      32618      0.015541  
      39105      0.017280  
      40457      0.018939  
      40925      0.019908  
      31903      0.013714  
      39937      0.011670  
      26040      0.012187  
      42199      0.011575  
      28432      0.015762  
      40340      0.020476  
      42407      0.018512  
      40301      0.007119  
      33346      0.015892  
      31617      0.008895  
      35309      0.009738  
      37740      0.008250  
      25546      0.014458  
      40899      0.014241  
      40587      0.003059  
      40535      0.005037  
      40665      0.005532  
      35400      0.002971